In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed,Dropout
from keras import backend as K
from keras.models import Model
from random import shuffle
from keras.optimizers import RMSprop,Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]    

In [3]:
def PREPROCESS(lines):    
    data_n = len(lines) - 1
    SEQ = np.zeros((data_n, 28, 4), dtype=int)
    Score = np.zeros((data_n, 1), dtype=float)
    shuffle(lines[1:])
    for l in range(1, data_n+1):
        data = lines[l].split(',')
        seq = data[6]
        Score[l-1] = float(data[4])
        for i in range(28):
            if seq[i] in "Aa":
                SEQ[l-1, i, 0] = 1
            elif seq[i] in "Cc":
                SEQ[l-1, i, 1] = 1
            elif seq[i] in "Gg":
                SEQ[l-1, i, 2] = 1
            elif seq[i] in "Tt":
                SEQ[l-1, i, 3] = 1
        #CA[l-1,0] = int(data[2])*100
	
    return SEQ, Score

In [4]:
FILE = open("/home/csgrads/dbais001/CRISP-CAS9/yeast_bact/yeast_data/yeast_kmcs_cont_w.csv", "r")
data = FILE.readlines()
FILE.close()
print(len(data))
SEQ, score = PREPROCESS(data)

34200


In [5]:
train_x = SEQ[0:20520,:]
train_y = score[0:20520]
print(train_x.shape)
print(train_y.shape)
val_x = SEQ[20520:27360,:]
val_y = score[20520:27360]
print(val_x.shape)
print(val_y.shape)
test_x = SEQ[27360:,:]
test_y = score[27360:]
print(test_x.shape)
print(test_y.shape)

(20520, 28, 4)
(20520, 1)
(6840, 28, 4)
(6840, 1)
(6839, 28, 4)
(6839, 1)


In [6]:
SEQ = Input(shape=(28,4))
blstm_1 = Bidirectional(LSTM(units=16, dropout=0.0,recurrent_dropout=0.0, return_sequences=True))(SEQ)
blstm_2 = Bidirectional(LSTM(units=8,dropout=0.0, return_sequences=True))(blstm_1)
#flatten = Flatten()(attn)
attn = AttentionWithContext()(blstm_2)
dropout_1 = Dropout(0.5)(attn)
dense_1 = Dense(80, activation='relu', kernel_initializer='glorot_uniform')(dropout_1)
dropout_2 = Dropout(0.5)(dense_1)
dense_2 = Dense(units=40,  activation="relu",kernel_initializer='glorot_uniform')(dropout_2)
dropout_3 = Dropout(0.3)(dense_2)
dense_3 = Dense(units=40,  activation="relu",kernel_initializer='glorot_uniform')(dropout_3)
out = Dense(units=1,  activation="linear")(dense_3)
model = Model(SEQ,out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 4)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 28, 32)            2688      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 28, 16)            2624      
_________________________________________________________________
attention_with_context_1 (At (None, 16)                288       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                1360      
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
__________

In [7]:
adam = Adam(lr = 0.001)
model.compile(loss='mean_squared_error', optimizer=adam)
checkpointer = ModelCheckpoint(filepath="cas9.hdf5",verbose=1, monitor='val_loss',save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
model.fit([train_x], train_y, batch_size=64, epochs=150, shuffle=True, validation_data=( [val_x], val_y), callbacks=[checkpointer,earlystopper])

Train on 20520 samples, validate on 6840 samples
Epoch 1/150
20520/20520 [==============================] - 12s - loss: 6.2066 - val_loss: 5.9173
Epoch 2/150
20520/20520 [==============================] - 11s - loss: 5.7609 - val_loss: 5.4183
Epoch 3/150
20520/20520 [==============================] - 11s - loss: 5.1525 - val_loss: 4.7813
Epoch 4/150
20520/20520 [==============================] - 11s - loss: 4.8196 - val_loss: 4.6735
Epoch 5/150
20520/20520 [==============================] - 11s - loss: 4.7070 - val_loss: 4.4555
Epoch 6/150
20520/20520 [==============================] - 11s - loss: 4.4831 - val_loss: 4.5654
Epoch 7/150
20520/20520 [==============================] - 11s - loss: 4.4478 - val_loss: 4.3371
Epoch 8/150
20520/20520 [==============================] - 11s - loss: 4.3315 - val_loss: 4.2358
Epoch 9/150
20520/20520 [==============================] - 11s - loss: 4.2707 - val_loss: 4.5640
Epoch 10/150
20520/20520 [==============================] - 11s - loss: 4.2248

In [11]:
import scipy.stats as st
print('testset')
pred_y = model.predict([test_x])
#print('mse ' + str(mse(test_y, pred_y)))
print(st.spearmanr(test_y, pred_y))
y_pred_tr = model.predict([train_x])
print(st.spearmanr(train_y, y_pred_tr))

testset
SpearmanrResult(correlation=0.6119674658660605, pvalue=0.0)
SpearmanrResult(correlation=0.6371169383494569, pvalue=0.0)


In [12]:
pred_y = pred_y.flatten()
test_y = test_y.flatten()
print(st.pearsonr(test_y, pred_y))
print(st.pearsonr(train_y.flatten(), y_pred_tr.flatten()))

(0.6636359350323923, 0.0)
(0.741786108220231, 0.0)
